# The Dealer class

This notebook provides some documentation about Dealer objects.

## Importing the Dealer class

The current working directory needs to be "fivecarddraw" in order to run the code cells of this notebook. Assuming the fivecarddraw repository has been cloned, then this notebook is contained within the "fivecarddraw/notebooks" folder. To change the current working directory, run the next cell.

In [1]:
from os import chdir, getcwd

if not getcwd().endswith("fivecarddraw"):
    chdir("..")
    
print(f"Current Directory: {getcwd()}")

Current Directory: d:\My Projects\Python\Games\fivecarddraw


Then the Dealer class can be imported in the usual pythonic way.

In [2]:
from fivecarddraw import Dealer

## Creating a Dealer

The Dealer class is a subclass of the Table class, which is essentially a list of Player objects, with a Deck and a pot. The Dealer class can therefore access any gamestate related data, to perform any actions it needs to. A consequence of the inheritence is that Dealers need to be initialised with a list of Player objects.  

In [3]:
from fivecarddraw import Player

players = [Player(f"Player {i+1}") for i in range(5)]
print(f"Players: {players}")

Players: [Player 1, Player 2, Player 3, Player 4, Player 5]


Initialising the dealer will take a moment as the dealer object will populate 3 quite large lookup tables. The lookup tables play a part in hand evaluation, which is outlined in more detail in the hand.ipynb notebook.

In [4]:
dealer = Dealer(players)

print(f"Players: {dealer}")
print(f"Deck: {dealer.deck}")

Players: [Player 1, Player 2, Player 3, Player 4, Player 5]
Deck: [2♠, 3♠, 4♠, 5♠, 6♠, 7♠, 8♠, 9♠, 10♠, J♠, Q♠, K♠, A♠, 2♡, 3♡, 4♡, 5♡, 6♡, 7♡, 8♡, 9♡, 10♡, J♡, Q♡, K♡, A♡, 2♢, 3♢, 4♢, 5♢, 6♢, 7♢, 8♢, 9♢, 10♢, J♢, Q♢, K♢, A♢, 2♣, 3♣, 4♣, 5♣, 6♣, 7♣, 8♣, 9♣, 10♣, J♣, Q♣, K♣, A♣]


## Tracking the Table Button 

The table button is tracked by maintaining a sequence of players, that mimics player seat positions clockwise around a table; and cycling the sequence, so that the player who has the button is last in the sequence. This means that turn order begins with the player at index 0 of the dealer.

In [5]:
dealer.MoveButton()
print(f"Players: {dealer}")
print(f"Button player: {dealer[-1]}")

Players: [Player 2, Player 3, Player 4, Player 5, Player 1]
Button player: Player 1


## Dealing, Evaluating and Collecting Hands

The way that Dealers deal hands mimics the way that casino dealers deal hands at casinos. The deck is shuffled. Then a card is given to each player starting from index 0, and then another card is dealt to each 4 more times. Evaluation and collection is outlined in the hand.ipynb notebook. 

In [25]:
dealer.ShuffleDeck()
dealer.DealHands()
rankings = dealer.EvaluateHands()

for i in range(len(dealer)):
    rating, hit = rankings[str(dealer[i])], dealer.Lookup(rankings[str(dealer[i])])
    print(f"{dealer[i]}'s hand: {dealer[i].hand} (The number {rating} best possible hand with {hit}.)")

dealer.CollectCards()

Player 2's hand: 3♣ 7♠ 4♡ K♢ 2♡ (The number 7002 best possible hand with high card.)
Player 3's hand: 6♢ 2♣ 2♢ J♣ 2♠ (The number 2436 best possible hand with three of a kind.)
Player 4's hand: J♢ A♢ 6♣ 8♠ 8♢ (The number 4668 best possible hand with pair.)
Player 5's hand: Q♡ 8♣ 3♠ 8♡ Q♢ (The number 2763 best possible hand with two pair.)
Player 1's hand: Q♠ J♡ 5♣ 4♢ 10♠ (The number 7029 best possible hand with high card.)


## Taking Bets

The Dealer class is given the responsibility to take bets from players and put them in the pot, and this is mainly to ensure that the bets are legal, and are communicated to everybody. The semantics of this process might look a little bit off, but the gameplay loop itself is a subclass of the Dealer class, so when these methods are used in an actual game by the FiveCardDraw object, the semantics make more sense. 

In [8]:
for player in dealer:
    player.chips += 100

dealer.ante = 10
dealer.TakeAnte()


Player 2 paid 10 chips for the ante.
Player 3 paid 10 chips for the ante.
Player 4 paid 10 chips for the ante.
Player 5 paid 10 chips for the ante.
Player 1 paid 10 chips for the ante.
